In [2]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
a = torch.arange(6).reshape(2, 3)
print(a)

b = a +1
print(b)

tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[1, 2, 3],
        [4, 5, 6]])


In [4]:
a0 = torch.tensor([1, 2, 3, 4])
a1 = torch.arange(8).reshape(2, 4)
print(a0)
print(a1)

tensor([1, 2, 3, 4])
tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])


In [5]:
torch.matmul(a1, a0)

tensor([20, 60])

In [6]:
# 行列の積（バッチ）

a0 = torch.arange(24).reshape(-1, 2, 4)
a1 = torch.arange(24).reshape(-1, 4, 2)

torch.matmul(a0, a1)

tensor([[[  28,   34],
         [  76,   98]],

        [[ 428,  466],
         [ 604,  658]],

        [[1340, 1410],
         [1644, 1730]]])

In [7]:
print(a0.dtype)
print(a0.type())


torch.int64
torch.LongTensor


In [8]:
a0 = torch.arange(24).reshape(-1, 2, 4).float()
print('a0の型：',a0.dtype)

c = a0.long()
print('cの型：', c.dtype)
print(f'c形成後のa0の型：{a0.dtype}') # 変化なし　castingはimmutable

a0の型： torch.float32
cの型： torch.int64
c形成後のa0の型：torch.float32


In [9]:
# torchとnumpyへの変換

a0 = torch.arange(3)
print(a0.dtype)

b0 = a0.numpy()
print(b0.dtype)

a1 = torch.from_numpy(b0)
print(a1.dtype)

torch.int64
int64
torch.int64


In [10]:
# 微分をもったtonsorのnumpy変換は不可 -> detch()で切り離す必要がある

a = torch.tensor([1.], requires_grad=True)
print(a)
print(a.dtype)

a.numpy()

tensor([1.], requires_grad=True)
torch.float32


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [11]:
a.detach().numpy()

array([1.], dtype=float32)

In [12]:
np.array([1.]).dtype

dtype('float64')

In [13]:
# tensorの結合
a = torch.zeros(6).reshape(2, 3)
b = torch.ones(6).reshape(2, 3)

# 基本はvstack
print(torch.cat((a, b)))

# hstack
print(torch.cat((a,b), dim=1))


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.]])


In [14]:
# 2次元の配列のリストをバッチ化
a = torch.zeros(6).reshape(2, 3)
b = torch.ones(6).reshape(2, 3)
c = a + 2

torch.stack([a, b, c])

tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.]],

        [[2., 2., 2.],
         [2., 2., 2.]]])

In [15]:
# 軸の追加

a = torch.zeros(6).reshape(2, 3)
b = a.unsqueeze(0)
print(a.shape)
print(b.shape)
print(b)

torch.Size([2, 3])
torch.Size([1, 2, 3])
tensor([[[0., 0., 0.],
         [0., 0., 0.]]])


In [16]:
# 軸の入れ替え
a = torch.arange(12).reshape(2, 2, 3)
print(a)

torch.permute(a, (2, 0, 1))


tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]]])


tensor([[[ 0,  3],
         [ 6,  9]],

        [[ 1,  4],
         [ 7, 10]],

        [[ 2,  5],
         [ 8, 11]]])

## 自動微分

In [17]:
x1 = torch.tensor([1.], requires_grad=True)
x2 = torch.tensor([2.], requires_grad=True)
x3 = torch.tensor([3.], requires_grad=True)

z = (x1 - 2*x2 -1)**2 + (x2*x3 - 1)**2 + 1

z.backward()
x1.grad

tensor([-8.])

In [18]:
def f(x):
    return (x[0]-2*x[1]-1)**2 + (x[1]*x[2]-1)**2 + 1


def f_grad(x):
    z = f(x)
    z.backward()
    return x.grad


x = torch.tensor([1., 2., 3., ], requires_grad=True)

for i in range(50):
    x = x - 0.1 * f_grad(x)
    x = x.detach().requires_grad_(True)

print(f"x = {x.data}, f = {f(x).item()}")

x = tensor([-0.2087, -0.6063, -1.6405]), f = 1.0000436305999756


## Pytorchの学習プログラムの作成

___
・基本はdatasetでデータ準備（呼び出し＋加工）、dataloaderでそのデータを呼び出す。  
・NLPではDataLoaderを使うのは面倒なのであまり使用しない？？

In [55]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# データ準備
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.5)

# 型の指定
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [56]:
# モデルの定義
class MyIris(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(4, 6)
        self.l2 = nn.Linear(6, 3)

    def forward(self, x):
        h1 = torch.sigmoid(self.l1(x))
        h2 = self.l2(h1)
        return h2        

In [57]:
model = MyIris()

optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()


In [103]:
# 順伝播
model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    print(epoch, loss.item())
    loss.backward()
    optimizer.step()


0 0.07907465845346451
1 0.07904881238937378
2 0.07902298867702484
3 0.0789971873164177
4 0.07897141575813293
5 0.07894568145275116
6 0.07891999185085297
7 0.07889430224895477
8 0.07886867225170135
9 0.07884306460618973
10 0.07881751656532288
11 0.07879196852445602
12 0.07876645773649216
13 0.07874096184968948
14 0.07871551811695099
15 0.07869008183479309
16 0.07866471260786057
17 0.07863935828208923
18 0.07861402630805969
19 0.07858874648809433
20 0.07856345176696777
21 0.0785382017493248
22 0.0785129964351654
23 0.078487828373909
24 0.07846266776323318
25 0.07843755930662155
26 0.0784124881029129
27 0.07838740944862366
28 0.07836239784955978
29 0.07833739370107651
30 0.078312449157238
31 0.07828748971223831
32 0.07826260477304459
33 0.07823772728443146
34 0.07821288704872131
35 0.07818806171417236
36 0.0781632736325264
37 0.07813850045204163
38 0.07811379432678223
39 0.07808908820152283
40 0.07806441932916641
41 0.07803979516029358
42 0.07801517844200134
43 0.07799059897661209
44 0.07

In [62]:
#モデルデータのweight paramsの保存
torch.save(model.state_dict(), 'myiris.model')

In [64]:
# 保存したパラメータのロード
model.load_state_dict(torch.load('myiris.model'))

<All keys matched successfully>

In [75]:
a = torch.load('myiris.model')

In [78]:
a

OrderedDict([('l1.weight',
              tensor([[ 0.0686,  1.3287, -1.5608, -1.1699],
                      [ 0.1092,  0.8625, -1.1287, -0.3739],
                      [ 1.4288,  1.9610, -2.4812, -2.4839],
                      [ 0.6149,  0.8108, -1.1207, -0.9686],
                      [-0.7722, -1.5642,  1.8016,  1.1260],
                      [-0.3825, -1.0168,  1.8788,  0.1965]])),
             ('l1.bias',
              tensor([ 0.4125,  0.3026,  1.4632,  0.1256, -0.7394,  0.0610])),
             ('l2.weight',
              tensor([[ 2.7636,  1.2256,  1.9447,  1.2721, -2.3186, -2.4064],
                      [-1.3091, -0.6852,  2.5964,  0.7811, -0.8011,  1.2268],
                      [-0.8778, -1.2536, -4.0985, -1.4169,  3.4784,  1.5488]])),
             ('l2.bias', tensor([-0.7360, -0.1329,  0.6786]))])

In [79]:
a['l2.bias']

tensor([-0.7360, -0.1329,  0.6786])

In [80]:
# 予測
model.eval()
torch.no_grad()

In [93]:
model.eval()
with torch.no_grad():
    output = model(X_test)
    pred = torch.argmax(output, axis=1)
    print((y_test == pred).sum()/ len(pred))

tensor(0.9733)


In [102]:
!python3 './pytorch-nlp01r1/Chapter1/iris0.py'

0 1.2748122215270996
1 1.2178274393081665
2 1.1747615337371826
3 1.1427992582321167
4 1.1194837093353271
5 1.1027058362960815
6 1.090729832649231
7 1.0821901559829712
8 1.076056957244873
9 1.0715770721435547
10 1.0682135820388794
11 1.06559157371521
12 1.0634511709213257
13 1.0616153478622437
14 1.0599639415740967
15 1.0584149360656738
16 1.056913137435913
17 1.0554208755493164
18 1.0539124011993408
19 1.0523701906204224
20 1.05078125
21 1.0491366386413574
22 1.0474295616149902
23 1.0456544160842896
24 1.0438060760498047
25 1.0418809652328491
26 1.0398740768432617
27 1.037781834602356
28 1.0355998277664185
29 1.0333229303359985
30 1.030946969985962
31 1.0284665822982788
32 1.0258771181106567
33 1.0231735706329346
34 1.020351767539978
35 1.0174078941345215
36 1.0143399238586426
37 1.011146903038025
38 1.0078306198120117
39 1.0043948888778687
40 1.0008474588394165
41 0.9971978664398193
42 0.9934588670730591
43 0.9896451830863953
44 0.9857724905014038
45 0.9818570017814636
46 0.9779136180

In [104]:
# ミニバッチ
n=75
bs=25

model.train()
for epoch in range(1000):
    idx = np.random.permutation(n) 
    for j in range(0, n, bs):
        X_batch = X_train[idx[j:(j+bs) if (j+bs) < n else n ]]
        y_batch = y_train[idx[j:(j+bs) if (j+bs) < n else n ]]
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        print('エポック数：', epoch, 'バッチ：',j ,'損失：', loss.item())
        loss.backward()
        optimizer.step()

エポック数： 0 バッチ： 0 損失： 0.04807636886835098
エポック数： 0 バッチ： 25 損失： 0.06510765850543976
エポック数： 0 バッチ： 50 損失： 0.08641405403614044
エポック数： 1 バッチ： 0 損失： 0.06647584587335587
エポック数： 1 バッチ： 25 損失： 0.02414884977042675
エポック数： 1 バッチ： 50 損失： 0.10337990522384644
エポック数： 2 バッチ： 0 損失： 0.07569265365600586
エポック数： 2 バッチ： 25 損失： 0.0831385925412178
エポック数： 2 バッチ： 50 損失： 0.07042722404003143
エポック数： 3 バッチ： 0 損失： 0.10264723002910614
エポック数： 3 バッチ： 25 損失： 0.06885568797588348
エポック数： 3 バッチ： 50 損失： 0.03971525654196739
エポック数： 4 バッチ： 0 損失： 0.06550800055265427
エポック数： 4 バッチ： 25 損失： 0.09876976162195206
エポック数： 4 バッチ： 50 損失： 0.03641676902770996
エポック数： 5 バッチ： 0 損失： 0.025826644152402878
エポック数： 5 バッチ： 25 損失： 0.08058851212263107
エポック数： 5 バッチ： 50 損失： 0.08322415500879288
エポック数： 6 バッチ： 0 損失： 0.05334346741437912
エポック数： 6 バッチ： 25 損失： 0.144711434841156
エポック数： 6 バッチ： 50 損失： 0.029244501143693924
エポック数： 7 バッチ： 0 損失： 0.10964407026767731
エポック数： 7 バッチ： 25 損失： 0.048448748886585236
エポック数： 7 バッチ： 50 損失： 0.05362822487950325
エポック数： 8 バッチ： 0 損失： 0.07

In [105]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [107]:
!python3 './pytorch-nlp01r1/Chapter1/iris2.py'

0 0 1.1324422359466553
0 25 1.1652086973190308
0 50 1.1835863590240479
1 0 1.1619246006011963
1 25 1.1377032995224
1 50 1.1369235515594482
2 0 1.1318548917770386
2 25 1.1249836683273315
2 50 1.1248546838760376
3 0 1.128318190574646
3 25 1.1204930543899536
3 50 1.1138103008270264
4 0 1.1135528087615967
4 25 1.1115400791168213
4 50 1.0973023176193237
5 0 1.0907284021377563
5 25 1.1135824918746948
5 50 1.0958540439605713
6 0 1.08332097530365
6 25 1.0950238704681396
6 50 1.0992827415466309
7 0 1.1034057140350342
7 25 1.0860646963119507
7 50 1.0758116245269775
8 0 1.0731257200241089
8 25 1.1017348766326904
8 50 1.0680454969406128
9 0 1.097965955734253
9 25 1.046463131904602
9 50 1.0785311460494995
10 0 1.050126552581787
10 25 1.0678000450134277
10 50 1.0801846981048584
11 0 1.0932668447494507
11 25 1.0626202821731567
11 50 1.0277585983276367
12 0 1.0536439418792725
12 25 1.0531282424926758
12 50 1.0513910055160522
13 0 0.9948280453681946
13 25 1.0923079252243042
13 50 1.0616624355316162
14 